In [1]:
import os
import sys
parent_dir = "/Midgard/home/farzantn/phd/Olfaction/MoLFormer_N2024"
sys.path.append(parent_dir)
parent_dir="/Midgard/home/farzantn/mambaforge/envs/MolTran_CUDA11_cuda/lib/python3.8"
sys.path.append(parent_dir)

In [2]:
from mat4py import loadmat
import scipy.io as sio
import numpy as np
parent_dir="/Midgard/home/farzantn/mambaforge/envs/MolTran_CUDA11_cuda/lib/python3.8"
sys.path.append(parent_dir)
import numpy as np
path='/local_storage/datasets/farzaneh/Olfaction/fmri_test/'
from utils.ridge_tools import *
import pandas as pd
import ast
base_directory = ''
from sklearn.model_selection import cross_validate,train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.metrics import roc_auc_score, mean_squared_error
import scipy

In [4]:
base_dir = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/'

In [6]:
parent_input_sagar_original = base_dir+'NEMO_scripts-master/Data/supporting_files/NEMO_s01/Fullmet_FIR.mat'
data_original = sio.loadmat(parent_input_sagar_original)

In [7]:
parent_input_sagar_original = base_dir+'NEMO_scripts-master/Data/odor_responses_S1-3_regionized/odor_responses_S1.mat'
data1 = sio.loadmat(parent_input_sagar_original)
parent_input_sagar_original = base_dir+'NEMO_scripts-master/Data/odor_responses_S1-3_regionized/odor_responses_S2.mat'
data2 = sio.loadmat(parent_input_sagar_original)
parent_input_sagar_original = base_dir+'NEMO_scripts-master/Data/odor_responses_S1-3_regionized/odor_responses_S3.mat'
data3 = sio.loadmat(parent_input_sagar_original)

In [4]:
# parent_input_sagar_original = 'NEMO_s01/task_struct.mat'
# sio.loadmat(parent_input_sagar_original)[].keys()

In [8]:
rois_all=  []
for data_s in [data1,data2,data3]:
    rois = {}
    rois['PirF']= data_s['odor_vals'][0][0]
    rois['PirT']= data_s['odor_vals'][0][1]
    rois['AMY']= data_s['odor_vals'][0][2]
    rois['OFC']= data_s['odor_vals'][0][3]
    rois_all.append(rois)


In [9]:
for rois in rois_all:
    for key in rois.keys():
        roi=rois[key]
        roi = np.moveaxis(roi,-1,0)
        # roi = np.mean(roi,-1)
        rois[key]=roi

In [7]:
# rois

In [8]:
def get_chemical_features_fixed_length(layer, seq_len, feat_type, feat_dir,  train_ind,test_ind):
    
    loaded = np.load( os.path.join(feat_dir, feat_type + '_length_'+str(seq_len)+ '_layer_' + str(layer) + '.npy') )
    # For now, all models should be processed the same way
    # In future, if there are newer models that are processed differently, can add additional if-branch 
    if True:
        train = loaded[train_indicator]         # (~3877, 768)
        test = loaded[~train_indicator]         # (~1279, 768)
    else:
        print('Unrecognized NLP feature type {}.'.format(feat_type))
    
    pca = PCA(n_components=10, svd_solver='full')
    pca.fit(train)
    train_pca = pca.transform(train)                                    # (~3877, 10)
    test_pca = pca.transform(test)                                      # (~1279, 10)

    return train, test, train_pca, test_pca 

In [9]:
def prepare_fmri_features(train_ind,test_ind):
        
    return tmp[train_ind], tmp[test_ind]

In [10]:
def CV_ind(n, n_folds):
    ind = np.zeros((n))                         # (1211,)
    n_items = int(np.floor(n/n_folds))          # 302
    for i in range(0,n_folds -1):               # Folds 0,1,2
        ind[i*n_items:(i+1)*n_items] = i
    ind[(n_folds-1)*n_items:] = (n_folds-1)     # Fold 3
    return ind    

In [11]:
def run_class_time_CV_fmri_crossval_ridge(data,
                                          regress_feat_names_list = [],method = 'kernel_ridge', 
                                          lambdas = np.array([0.1,1,10,100,1000]),
                                          detrend = False, n_folds = 4, skip=5):
    
    # nlp_feat_type = predict_feat_dict['nlp_feat_type']
    # feat_dir = predict_feat_dict['nlp_feat_dir']
    # layer = predict_feat_dict['layer']
    # seq_len = predict_feat_dict['seq_len']
        
    
    n_odors = data.shape[0]     # (1211)
    n_voxels = data.shape[1]    # (~27905)

    ind = CV_ind(n_odors, n_folds=n_folds)      # (1211,) => [0., 0., 0., ..., 3., 3., 3.]


    corrs = np.zeros((n_folds, n_voxels))
    acc = np.zeros((n_folds, n_voxels))
    acc_std = np.zeros((n_folds, n_voxels))

    all_test_data = []
    all_preds = []
    
    
    for ind_num in range(n_folds):
        train_ind = ind!=ind_num                # (1211,) => [False, False, False, ...,  True,  True,  True]
        test_ind = ind==ind_num                 # (1211,) => [ True,  True,  True, ..., False, False, False]
        
        
        _,_,tmp_train_features,tmp_test_features = get_chemical_features_fixed_length(layer, seq_len, nlp_feat_type, feat_dir, train_ind,test_ind)
        fmri_train_features,fmri_test_features = prepare_fmri_features(train_ind,test_ind)
        
        
        start_time = tm.time()
        weights, chosen_lambdas = cross_val_ridge(fmri_train_features,chemical_train_data, n_splits = 10, lambdas = np.array([10**i for i in range(-6,10)]), method = 'plain',do_plot = False)

        preds = np.dot(test_features, weights)
        corrs[ind_num,:] = corr(preds,test_data)
        all_preds.append(preds)
            
        print('fold {} completed, took {} seconds'.format(ind_num, tm.time()-start_time))
        del weights

    return corrs, acc, acc_std, np.vstack(all_preds), np.vstack(all_test_data)

In [12]:
def compute_correlation_keller(times , n_components,per_descritor=False):
    
    input_file_keller_pom = base_path+'alignment_olfaction_datasets/data/curated_datasets/embeddings/pom/keller_pom_embeddings_Apr17.csv'
    df_keller_mse_pom, df_keller_cor_pom = pipeline('pom',input_file_keller_pom,times=times,n_components=n_components)
    
    input_file_keller_alva = base_path+'alignment_olfaction_datasets/data/curated_datasets/alva/keller_molecules_alva_17Apr.csv'
    df_keller_mse_alva, df_keller_cor_alva = pipeline('alva',input_file_keller_pom,input_file_keller_alva,times=times,n_components=n_components)
    
    

    
    corrs_molformer=[]

    mses_molformer=[]
    
    for i in [0,1,2,3,4,5,6,7,8,9,10,11,13]:
    # for i in [0,13]:
        input_file_keller_molformer = base_path+'alignment_olfaction_datasets/data/curated_datasets/embeddings/molformer/keller_molformer_embeddings_'+str(i)+'_Apr17.csv'
        df_keller_mse_molformer, df_keller_cor_molformer = pipeline('molformer',input_file_keller_molformer,times=times,n_components=n_components,per_descritor=per_descritor)
        
        
        
        corrs_molformer.append(df_keller_cor_molformer)
        mses_molformer.append(df_keller_mse_molformer)

    return corrs_molformer,mses_molformer,df_keller_cor_pom,df_keller_mse_pom,df_keller_cor_alva,df_keller_mse_alva

In [13]:
def prepare_dataset(ds):
    # ds['y'] = ds['y'].apply(ast.literal_eval)
    ds['embeddings'] = ds['embeddings'].apply(ast.literal_eval)
    
    return ds

In [14]:
def custom_linear_regression(X,y):
    # print(y.shape)
    clf = RidgeCV(alphas=[1e-3, 1e-2, 1e-1, 1])
    # if len(y.shape)>1:
    #     linreg = MultiTaskLassoCV(max_iter=1000,n_alphas=200,random_state=seed,n_jobs=-1)
    # else:
    #     linreg = LassoCV(max_iter=1000,n_alphas=200,random_state=seed,n_jobs=-1)
    
    estimator= clf.fit(X,y)
    return estimator

In [15]:
def average_sagar_subject(df):
    df_groupbyCID=df.groupby(['CID','subject'])[['0.1',
     '1.1',
     '2.1',
     '3.1',
     '4.1',
     '5.1',
     '6.1',
     '7.1',
     '8.1',
     '9.1',
     '10.1',
     '11.1',
     '12.1',
     '13.1',
     '14.1'
]].mean().reset_index()

    df_groupbyCID['y'] = df_groupbyCID.loc[:, '0.1':'14.1'].values.tolist()
    df_embeddings=df.drop_duplicates(subset=['CID'])
    df_embeddings=df_embeddings[['CID','embeddings']]
    df_groupbyCID = pd.merge(df_groupbyCID, df_embeddings, on='CID', how='left')
    return df_groupbyCID

In [16]:
def average_sagar(df):
    df_groupbyCID=df.groupby('CID')[['0.1',
     '1.1',
     '2.1',
     '3.1',
     '4.1',
     '5.1',
     '6.1',
     '7.1',
     '8.1',
     '9.1',
     '10.1',
     '11.1',
     '12.1',
     '13.1',
     '14.1'
     ]].mean().reset_index()

    df_groupbyCID['y'] = df_groupbyCID.loc[:, '0.1':'14.1'].values.tolist()
    df_embeddings=df.drop_duplicates(subset=['CID'])
    df_embeddings=df_embeddings[['CID','embeddings']]
    df_groupbyCID = pd.merge(df_groupbyCID, df_embeddings, on='CID', how='left')
    return df_groupbyCID

In [17]:
# input_file_sagar_pom = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/embeddings/pom/sagar_pom_embeddings_Apr17.csv'
# pom_embeddings = pd.read_csv(input_file_sagar_pom)

In [18]:
parent_input_sagar = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/mols_datasets/curated_sagar_subjects_nonaminus.csv'
sagar_mols = pd.read_csv(parent_input_sagar)


In [19]:
parent_input_sagar = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/mols_datasets/curated_sagar_subjects_nonaminus.csv'
sagar_mols = pd.read_csv(parent_input_sagar)

In [20]:
# sagar_mols1 = sagar_mols.loc[sagar_mols['subject']==1].sort_values('cid')
# sagar_mols2 = sagar_mols.loc[sagar_mols['subject']==2].sort_values('cid')
# sagar_mols3 = sagar_mols.loc[sagar_mols['subject']==3].sort_values('cid')

sagar_mols.loc[sagar_mols['subject']==1].cid.values.tolist()
sagar_mols2 = sagar_mols.loc[sagar_mols['subject']==2]
sagar_mols3 = sagar_mols.loc[sagar_mols['subject']==3]

In [21]:
# rois_all[0]['PirF'].shape
# pd.DataFrame.from_dict(rois_all[0])
# selected_subject=sagar_mols[(sagar_mols['subject']==1)][['cid','IsomericSMILES','nonStereoSMILES']]
# selected_subject

In [22]:
def pipeline_regression(X,y,times,n_components=None):
    mserrorrs_corssvalidated = []
    correlations_corssvalidated = []
    for i in range(times):
       
        # print("min", X.min())
        # print("max", X.max())
          
        
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=seed+i)  
        scaler = StandardScaler()
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
        if n_components is not None:
            pca = PCA(n_components=n_components)
            X_train=pca.fit_transform(X_train)
            X_test=pca.transform(X_test)
        linreg =custom_linear_regression(X_train,y_train)
        mseerrors, correlations=metrics_per_descritor(X_test,y_test,linreg)
        mserrorrs_corssvalidated.append(mseerrors)
        correlations_corssvalidated.append(correlations)
    return mserrorrs_corssvalidated, correlations_corssvalidated

In [23]:
def metrics_per_descritor(X,y,linreg):
    predicted=linreg.predict(X)
    mseerrors = []
    correlations = []
    if len(y.shape)>1:
        for i in range(y.shape[1]):
            mseerror=mean_squared_error(predicted[:,i],y[:,i])
            correlation=scipy.stats.pearsonr(predicted[:,i], y[:,i])
            mseerrors.append(mseerror)
            correlations.append(correlation)
            # print(predicted[:,i], y[:,i])
        
    else:
        mseerror=mean_squared_error(predicted,y)
        correlation=scipy.stats.pearsonr(predicted, y)
        mseerrors.append(mseerror)
        correlations.append(correlation)
     # print(predicted[:,i], y[:,i])
    
    return mseerrors, correlations
        # plot()
        
    
    

In [24]:
def compute_correlation_keller(subject,roi,times , n_components,per_descritor=False):
    
    # input_file_keller_pom = base_path+'alignment_olfaction_datasets/data/curated_datasets/embeddings/pom/keller_pom_embeddings_Apr17.csv'
    # df_keller_mse_pom, df_keller_cor_pom = pipeline('pom',input_file_keller_pom,times=times,n_components=n_components)
    
    # input_file_keller_alva = base_path+'alignment_olfaction_datasets/data/curated_datasets/alva/keller_molecules_alva_17Apr.csv'
    # df_keller_mse_alva, df_keller_cor_alva = pipeline('alva',input_file_keller_pom,input_file_keller_alva,times=times,n_components=n_components)
    
    # selected_subject=sagar_mols[(sagar_mols['subject']==subject)][['cid','IsomericSMILES','nonStereoSMILES']]
            
    fmri_data_y=rois_all[subject-1][roi]
    # input_file_sagar_pom = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/embeddings/pom/sagar_pom_embeddings_Apr17.csv'
    

#X: representations
#y: fMRI
    
    corrs_molformer=[]

    mses_molformer=[]
    fmri_data_y_mean = fmri_data_y.mean(-1)
    for i in [0,1,2,3,4,5,6,7,8,9,10,11,13]:
    # for i in [0,1,2,3,4,13]:
        df_keller_mse_molformers=[]
        df_keller_cor_molformers=[]
        for y_i in range(fmri_data_y_mean.shape[1]): 
        # for y_i in range(7): 
            input_file_keller_molformer = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/embeddings/molformer/sagar_molformer_embeddings_'+str(i)+'_Apr17.csv'
            df_embeddings = pd.read_csv(input_file_keller_molformer)
            # df_embeddings_group = average_sagar_subject(df_embeddings)
            df_embeddings = prepare_dataset(df_embeddings)
            embeddings=df_embeddings[df_embeddings['subject']==subject]['embeddings']
            embeddings  = np.asarray(embeddings.values.tolist())
            
            
            df_keller_mse_molformer, df_keller_cor_molformer = pipeline_regression(embeddings,fmri_data_y_mean[:,y_i],times,n_components=n_components)
            df_keller_mse_molformers.append(df_keller_mse_molformer)
            df_keller_cor_molformers.append(df_keller_cor_molformer)
        
    corrs_molformer.append(df_keller_cor_molformers)
    mses_molformer.append(df_keller_mse_molformers)

    return corrs_molformer,mses_molformer

In [25]:
# # input_file_keller_molformer = '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/embeddings/molformer/sagar_molformer_embeddings_'+str(0)+'_Apr17.csv'
# d_test = pd.read_csv(input_file_keller_molformer)
# d_test['embeddings'].iloc[0]
# d_test[d_test['subject']==1]['embeddings']

In [26]:
# d_test[d_test['subject']==2]['embeddings']

In [27]:
# d_test[d_test['subject']==3]['embeddings']

In [28]:
def convert_fmri_data(data):
    datas=[]
    
    for data in data_list:
        array_str=data
        array_str = array_str.replace("[array(", "").replace("\n", "").replace(")]", "")
        array_np = np.array(ast.literal_eval(array_str))
        datas.append(array_np)
    # datas.shape
    return np.asarray(datas)
# Evaluating the string as a Python expression and converting it into a numpy array
# array_np
# data=np.array(selected_subject['OFC'].to_list())
# data.shape
# selected_subject['OFC'].to_list()

In [29]:
# input_files = ['/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/embeddings/pom/sagar_pom_embeddings_Apr17.csv',
#               '/local_storage/datasets/farzaneh/alignment_olfaction_datasets/curated_datasets/embeddings/molformer/sagar_molformer_embeddings_Apr17.csv'
#               ]

In [30]:
# # corrs_molfomers_a = np.asarray(corrs_molfomers) 
# corrs_molfomers_squeezed = np.squeeze(np.asarray(corrs_molfomers[0]),0)
# corrs_molfomers_squeezed = np.squeeze(corrs_molfomers_squeezed,-1)
# corrs_molfomers_squeezed
# 6,1,101,10,1

In [ ]:
seed=2024

# def extract_x_y(subject,area,input)\
corrs_molfomers=[]
mses_molformers=[]
results_df_all = pd.DataFrame()

for subject in [2,3]:
    for roi in ['OFC']:

# for subject in [1,2]:
    # for roi in ['PirF','PirT','OFC']:
        corrs_molfomer,mses_molformer = compute_correlation_keller(subject=subject,roi=roi,times=30 ,n_components=20,per_descritor=False)

        corrs_molfomer_squeezed = np.squeeze(np.asarray(corrs_molfomer),0)
        corrs_molfomer_squeezed = np.squeeze(corrs_molfomer_squeezed,-2)
         # corrs_molfomers.append(corrs_molfomer_squeezed)   
        
        mses_molfomer_squeezed = np.squeeze(np.asarray(mses_molformer),0)
        mses_molfomer_squeezed = np.squeeze(mses_molfomer_squeezed,-1)
    
        # data_dict = {'mse': mses_molfomer_squeezed , "corr": corrs_molfomer_squeezed}
        # data_df = pd.DataFrame.from_dict(data_dict)
        # data_df.to_csv(roi +"_"+ str(subject)+ "fmri.csv")
        statistic= corrs_molfomer_squeezed[:,:,0]
        p_value= corrs_molfomer_squeezed[:,:,1]
        p_value_df = pd.DataFrame.from_dict(p_value)
        p_value_df['fmri_index'] = p_value_df.index
        p_value_df['data'] = "p_value"
        statistics_df = pd.DataFrame.from_dict(statistic)
        statistics_df['fmri_index'] = statistics_df.index
        statistics_df['data'] = "statistic"
        mses_df = pd.DataFrame.from_dict(mses_molfomer_squeezed)
        mses_df['fmri_index'] = mses_df.index
        mses_df['data'] = "mse"
        results_df = pd.concat([p_value_df, statistics_df])
        results_df = pd.concat([results_df, mses_df])
        results_df["roi"] = roi
        results_df["subject"] = subject
        results_df.to_csv('fmri_'+str(subject)+'_'+roi+'.csv', index=False)  
        results_df_all = pd.concat([results_df, results_df_all])
        


    

    
    # mses_molformers.append(mses_molfomer_squeezed)   

        

In [ ]:
results_df_all.to_csv('fmri_all.csv', index=False)  

In [ ]:
# corrs_molfomer_squeezed = np.squeeze(np.asarray(corrs_molfomer),0)
# corrs_molfomer_squeezed = np.squeeze(corrs_molfomer_squeezed,-2)
#  # corrs_molfomers.append(corrs_molfomer_squeezed)   

# mses_molfomer_squeezed = np.squeeze(np.asarray(mses_molformer),0)
# mses_molfomer_squeezed = np.squeeze(mses_molfomer_squeezed,-1)

# data_dict = {'mse': mses_molfomer_squeezed , "corr": corrs_molfomer_squeezed}
# data_df = pd.DataFrame.from_dict(data_dict)
# data_df.to_csv(roi +"_"+ str(subject)+ "fmri.csv")

In [ ]:
np.asarray(mses_molformer).shape 
